In [1]:
!pip install librosa soundfile numpy sklearn 

In [2]:
!pip install pipwin
!pipwin install pyaudio

Package `pyaudio` found in cache
https://download.lfd.uci.edu/pythonlibs/w6tyco5e/PyAudio-0.2.11-cp38-cp38-win_amd64.whl
PyAudio-0.2.11-cp38-cp38-win_amd64.whl
[*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [------------------] [0.0%, 0s left]   [*] 0 bytes / 110 kB @ 0 bytes/s [--------------

In [3]:
import librosa
import soundfile

In [4]:
import numpy as np
import pandas as pd


In [5]:
# we need to use three features of a sound file, which are mfcc:Mel Frequency Cepsral Coefficient
# which represents the short-term power spectrum of a sound 
# about mfccc:wikipedia link - https://en.wikipedia.org/wiki/Mel-frequency_cepstrum 
# 2nd feature is chroma: pertains to 12 different pitch classes
# 3rd feature is mel: Mel spectrogram Frequency


In [6]:
# we need some to import some modules necessary to train a model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [7]:
# we are going to use neural network for detecting stress or mood of a user
# we are going to use MLPclassifier
from sklearn.neural_network import MLPClassifier


In [8]:
# so we need to extract those three features from a sound file which are mfcc,chroma,mel
# first parameter is the sound file name, and the rest three are boolean parameters 
def function_extracting_features(file,mfcc,chroma,mel):
    with soundfile.SoundFile(file) as file_sound: 
        read_sound_file = file_sound.read(dtype = 'float32')
        sample_rate = file_sound.samplerate 
        final = np.array([])
        if chroma:
            # if chroma is true then, we need to get the short time fourier transformation of read_sound_file
            short_tft = np.abs(librosa.stft(read_sound_file))
        if mfcc:
            # if mfcc is true then call the librosa.feature.mfcc and get mean value of it
            mfccs = np.mean(librosa.feature.mfcc(y=read_sound_file, sr=sample_rate, n_mfcc=40).T, axis=0)
            final = np.hstack((final, mfcss))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=short_tft, sr=sample_rate).T, axis=0)
            final = np.hstack((final,chroma))

        if mel:
            mel_new = np.mean(librosa.feature.melspecrogram(read_sound_file, sr=sample_rate).T, axis=0)
            final = np.hstack((final, mel_new))

        return final
        
    
        
        

In [9]:
# now we will create a dictionary to hold numbers and emotions that are present in ravdess dataset

feelings = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}
# we are going to observe only stressful emotions
observed_feelings = ['sad','angry','fearful']

In [10]:
# now we should load the ravdess data and the number in the feelings dictionary and the data files 
# contain the number with dashed in between eg: 01-02-01-07.wvl
import os,glob,pickle
def load_data_from_dataset(test_size=0.2):
    X=[]
    Y=[]
    for sound_file in glob.glob("E:\vjhackaton\VJH_012\dataset\Actor_*\\*.wav"):
        sound_file_name = os.path.basename(sound_file)
        feeling = feelings[sound_file_name.split("-")[2]]
        if feeling not in observed_feelings:
            continue
        feature = function_extracting_features(sound_file_name, mfcc=True,chroma=True,mel=True)
        X.append(feature)
        Y.append(feeling)
    return train_test_split(np.array(X),Y,test_size=test_size,random_state=9)

        
        
        
        

In [11]:
# splitting the dataset into train and test 
x_train,y_train,x_test,y_test = load_data_from_dataset(0.2)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.